In [1]:
import os

In [2]:
%pwd

'd:\\aiproject\\mydeeplearning_project\\research'

In [3]:
os.chdir('../')

In [4]:
%pwd

'd:\\aiproject\\mydeeplearning_project'

In [6]:
# dagshub 
# mlflow >> used to track ML training experiment

# model >> lr >> 0..01 >> acc
# model >> lr >> 0.001 >> acc

In [7]:
#  https://dagshub.com/surajkale99/mydeeplearning_project.mlflow

In [9]:
#!pip install dagshub

In [5]:
import dagshub
dagshub.init(repo_owner='surajkale99', repo_name='mydeeplearning_project', mlflow=True)

import mlflow
with mlflow.start_run():
  mlflow.log_param('parameter name', 'value')
  mlflow.log_metric('metric name', 1)

Accessing as surajkale99

Initialized MLflow to track repo "surajkale99/mydeeplearning_project"

Repository surajkale99/mydeeplearning_project initialized!

c:\Users\Admin\anaconda3\envs\py310\lib\site-packages\mlflow\utils\requirements_utils.py:12: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


In [6]:
os.environ["MLFLOW_TRACKING_URI"]="https://dagshub.com/surajkale99/mydeeplearning_project.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"]="surajkale99"
os.environ["MLFLOW_TRACKING_PASSWORD"]="e4f2406488322cebc687e5c389841eb67f8beeec"

In [7]:
mlflow.set_tracking_uri("https://dagshub.com/surajkale99/mydeeplearning_project.mlflow")

In [14]:
%pwd

'd:\\aiproject\\mydeeplearning_project'

In [10]:
import tensorflow as tf
model = tf.keras.models.load_model("artifacts/training/model.keras")

In [14]:
# entity 

In [8]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    all_params: dict
    mlflow_uri: str
    params_image_size: list
    params_batch_size: int

In [9]:
# configuration manager

In [11]:
from src.cnnClassifier.constants import *
from src.cnnClassifier.utils.common import read_yaml,create_directories,save_json


class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    
    def get_evaluation_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
            path_of_model="artifacts/training/model.keras",
            training_data="artifacts/data_ingestion/kidney-ct-scan-image",
            mlflow_uri="https://dagshub.com/surajkale99/mydeeplearning_project.mlflow",
            all_params=self.params,
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE
        )
        return eval_config

In [12]:
#components

In [ ]:
import tensorflow as tf
from pathlib import Path
import mlflow
import mlflow.keras
from urllib.parse import urlparse


class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config

    
    def _valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.30
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )


    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)
    

    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self._valid_generator()
        self.score = model.evaluate(self.valid_generator)
        self.save_score()

    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path=Path("scores.json"), data=scores)

            
                
    def log_into_mlflow(self):
        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme

        with mlflow.start_run():
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics({
            "loss": self.score[0],
            "accuracy": self.score[1]
        })

        # Save model explicitly with .keras extension
        model_path = "artifacts/training/model.keras"
        self.model.save(model_path)

        # Log as artifact
        mlflow.log_artifact(model_path)

        # Register model only if not file store
        if tracking_url_type_store != "file":
            mlflow.register_model(
                "runs:/{}/model.keras".format(mlflow.active_run().info.run_id),
                "VGG16Model"
            )


In [16]:
# pipeline

In [21]:
try:
    config = ConfigurationManager()
    eval_config = config.get_evaluation_config()
    evaluation = Evaluation(eval_config)
    evaluation.evaluation()
    evaluation.log_into_mlflow()
except Exception as e:
   raise e

[2026-01-12 13:11:33,563: INFO: common: yaml file: config\config.yaml loaded successfully]
[2026-01-12 13:11:33,570: INFO: common: yaml file: params.yaml loaded successfully]
[2026-01-12 13:11:33,573: INFO: common: created directory at: artifacts]
Found 139 images belonging to 2 classes.
9/9 ━━━━━━━━━━━━━━━━━━━━ 56s 6s/step - accuracy: 0.9712 - loss: 0.1050
[2026-01-12 13:12:30,673: INFO: common: json file saved at: scores.json]


Successfully registered model 'VGG16Model'.
2026/01/12 13:13:10 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: VGG16Model, version 1
Created version '1' of model 'VGG16Model'.
